In [13]:
import pandas as pd
import numpy as np
from datetime import datetime
import pickle
import os.path

import google
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

In [14]:

scopes = ['https://www.googleapis.com/auth/spreadsheets',
          'https://www.googleapis.com/auth/drive']

credentials = Credentials.from_service_account_file("client_secrets1.json", scopes=scopes)

gc = gspread.authorize(credentials)

gauth = GoogleAuth()
drive = GoogleDrive(gauth)

# Open a .gsheet
gs = gc.open_by_key("1-ahvkjhkjhvjdsvkjs kjhkjfhkahfkjhkjhkjah")

# Select a work sheet from its name
worksheet1 = gs.worksheet("Sales")

In [15]:
# Pull Data From .gsheets
data_pull = "Shopify!A1:H10000"

Sheet_Dict = {"Ashley" : "11rM_IdQgTp-C4OAby2FjA3jCF3kmQcrMl2NfliySeMQ",
              "Bellona" : "1tk7zeUWyqSckbe7QlCIoMwupz3wkps7NyDZy2Zdc6Lg",
              "Beyan" : "13h3mS2zCL_hkOC4P0pQTvjhPa7Z95t0lR6o8rDN1HBE",
              "Casamode" : "1CpRiANh-Bu0zJfZ2Y4EFLoGAJBExFtWrznACEeRzYDU",
              "Coaster" : "1SXuwwYXMWaUXDuwKe_uIaNN8OumGMwoyDjhjbfv19PM",
              "Corsicana" : "1wEWOO1hZtkaMnKoKswKyi1_dWs4hE4PV4SfSd0_strw",
              "Cosmos" : "1lryk9YFXmEjmAhSAJhHZgDm7WjBncQKirkJ291dzYsI",
              "Crown_Mark" : "1uL39iYxNWcjsHHiq9dcMD4q7wr5etM2mmWlZpmJ_hu0",
              "Donco_Kids" : "1m-UXoA0hMQgdQC7i5kZieGWiYWkuITqBZqpSEi5QzUI",
              "Empire" : "1wqMMvNcRWl-0ym_Bn6PR5Deu-0CvhgllrLsBn9QH1nw",
              "Furniture_of_America" : "1wdNt0oPuZVgtLyN6zA9LwRQQvipLF1Tj6vmsFj0FFMA",
              "Furniture_World" : "1s5CImHPUm5H1O48a0p_e4Ph5YWYPnXT6CNRV_fYXKpU",
              "Galaxy" : "1p6wkUt8baI8BAVbg7n6qa2Ct-wh1v4aZeWyd5hG53RM",
              "Global_Furniture" : "1MGYKdxPbVMrEtaL_reSplUqXnh_jzNHnmJtpZV2k4cM",
              "Happy_Homes" : "1g9ut7lXhDiD9yeW9-h_81SEsres9cPfqcTKxcACfA6Q",
              "Homelegance" : "129PWgvIp4qE56r-1--0ntuGwxnS0kCkl591jcV8mcV8",
              "Lidixh" : "1GtdxvH03AfjqcrknDxVB966CIYrRWP1V2MsYID3cpLU",
              "Mainline" : "1qPewlXYGcJDLUpdzDFfLRjriMUO9HLyE6rGHJv96iwo",
              "Meridian" : "1Pzb9kcgd3_FT3iLsB0yVAkyBGL2kNQj3OwPhEjS_SAU",
              "New_Art" : "1KpgUz0VU3NnSmHGhWKimpNFu1b5o2atqmTZyxP49Mrg",
              "New_Classic" : "17ZG-sQDOn7V4UbmoWl4caStytimdaeL5TSM074y7SYQ",
              "New_York_Diamond" : "1y2rwb9529J1u0Zgqn-nYacjw9gdVM0ezaM4fcI-hxJQ",
              "Nova" : "19cu7_yFO7LWdA5XvfhxLdjeQzy6nKpmfpmSq4oDjYZ0"}

Vendor_List = list(Sheet_Dict.keys())
Sheet_List = list(Sheet_Dict.values())

In [16]:
# Google API Check Function
def gsheet_api_check(SCOPES):
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secrets.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

In [17]:
# Google Sheet Data Pull Function
def pull_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_PULL):
    creds = gsheet_api_check(SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=DATA_TO_PULL).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=DATA_TO_PULL).execute()
        data = rows.get('values')
        #print("COMPLETE: Data copied")
        return data

In [18]:
# Connect to Google Drive
gsheet_api_check(scopes)

In [19]:
# Concat All .gsheets to One .gsheet

sheet = "13yL-WmxrqPGZqHi7ba1MI_UcXqd1F-I1Jb4bNrMew8s"
data_sheets = ["Shopify!A1:AX10000","ZarMoney!A1:AX10000"]
sales_list = ["Shopify","ZarMoney"]
df_all = pd.DataFrame()

i = 0
for data_sheet in data_sheets:
    sale = sales_list[i]
    data = pull_sheet_data(scopes, sheet, data_sheet)
    df = pd.DataFrame(data[1:], columns=data[0])
    df = df.iloc[:df.iloc[:,0].replace('', np.nan).count()]
    print(f"{sale} copied & shape: {df.shape}")  
    df_all = pd.concat([df_all, df])
    i +=1
print(f"All copied & shape: {df_all.shape}")
df = df_all
df.dropna(axis=0,how="all",inplace=True)

Shopify copied & shape: (2383, 50)
ZarMoney copied & shape: (1414, 50)
All copied & shape: (3797, 50)


In [20]:
# DataFrame to .gsheet
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe=df, include_index=False, include_column_header=True, resize=True)